In [ ]:
!pip install openai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
client = OpenAI(api_key="본인의 키를 입력하세요.")

In [ ]:
import pandas as pd
import json

# Excel 파일 읽기
df = pd.read_excel('poems3.xlsx')

# JSONL 파일로 저장할 함수 정의
def write_jsonl(filename, data):
    with open(filename, 'w', encoding='utf-8') as f:
        for item in data:
            json_record = json.dumps(item, ensure_ascii=False)
            f.write(json_record + '\n')

# 결과를 저장할 리스트
dialogue_pairs = []

# 각 행에 대해 대화쌍 생성
for _, row in df.iterrows():
    pair = {
        "messages": [
            {
                "role": "system",
                "content": "당신은 한국어로 시를 쓰는 시인입니다."
            },
            {
                "role": "user",
                "content": f"{row['소재']} 있는데 이걸로 시 한 편 써주세요."
            },
            {
                "role": "assistant",
                "content": row['시'],
                "weight": 1
            }
        ]
    }
    dialogue_pairs.append(pair)

# JSONL 파일로 저장
write_jsonl('poem_dialogue.jsonl', dialogue_pairs)

print("변환이 완료되었습니다. 결과가 'poem_dialogue.jsonl' 파일에 저장되었습니다.")

변환이 완료되었습니다. 결과가 'poem_dialogue.jsonl' 파일에 저장되었습니다.


In [ ]:
# 파인튜닝용 파일 생성하기
input_file = client.files.create(
  file=open("poem_dialogue.jsonl", "rb"),
  purpose="fine-tune"
)

In [ ]:
BATCH_SIZE = 1
EPOCHS = 3
LEARNING_RATE = 1.8

ft_job = client.fine_tuning.jobs.create(
  training_file=input_file.id,
  model="gpt-4o-mini-2024-07-18",
  hyperparameters={"batch_size":BATCH_SIZE,
                   "n_epochs":EPOCHS,
                   "learning_rate_multiplier":LEARNING_RATE,
                   },
#   hyperparameters={"batch_size":1,
#                    "n_epochs":3,
#                    "learning_rate_multiplier":1.8,
#                    }, # 훈련 반복 횟수를 정해줄 수 있음. 안하면 auto(여기서는 10개의 데이터를 2바퀴 돈다는 뜻)
  suffix=f"mya_{BATCH_SIZE}_{EPOCHS}_{LEARNING_RATE}"
)

In [ ]:
# 이 코드를 갱신해야 아래 것도 갱신이 됨.
ft_info = client.fine_tuning.jobs.retrieve(ft_job.id)
ft_info.id

'ftjob-A4k3KhvCkRmQIxvV2PtiSVMr'

In [ ]:
ft_info.status

'succeeded'

In [ ]:
stuff = input("소재를 입력해주세요 : ")

stream = client.chat.completions.create(
      model=ft_info.fine_tuned_model,
      #model='ft:gpt-4o-mini-2024-07-18:personal:mya-2:A4I6RYvQ',
      #model='ft:gpt-4o-mini-2024-07-18:personal::A49k3hw3',
      temperature=0.9,
      top_p=0.97,
      stream=True,
      messages=[
        {"role": "system", "content": "당신은 한국어 시를 쓰는 시인입니다."},
        {"role": "user", "content": f"'{stuff}' 소재로 시 한 편 써주세요."}
      ]
    )

print("-----------poem-----------")

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

소재를 입력해주세요 : 개미굴
-----------poem-----------
개미굴에 들어가면

눈앞이 어두워지는 것이 좋다

오랜만에 출구를 발견하고도

들어온 입구로 돌아가는 것이 좋다

개미굴은 개미만 다니는 곳이다

개미굴에서는 말하지 않는 것이 좋다

외부의 소음이 차단되는 것 같아

개미굴에서 불을 피우지 않는 것이 좋다

주변이 환해지면

주변을 돌아보게 된다

개미굴에서는 뒤를 돌아보지 않는 것이 좋다

개미굴은 굴다

굴은 길이 없다고 생각하는 것이 좋다

마지막에 내는 소리를 생각하며

개미굴에 들어서는 것이 좋다

쓰고, 버리고, 지우는

일련의 행위가 개미굴이라면

개미굴에 들어가면 각서 한 장 없이

앞으로의 행위를 약속하게 된다

개미굴 안에서는

사람이 되어서는 안 되는 것이다

In [ ]:
stuff = client.chat.completions.create(
      model=ft_info.fine_tuned_model,
      temperature=0.8,
      top_p=0.97,
      messages=[
        {"role": "system", "content": "당신은 랜덤으로 한국어 단어를 하나 고르는 AI입니다."},
        {"role": "user", "content": "단어 하나를 골라주세요."}
      ]
    )

stuff = stuff.choices[0].message.content
print(f"소재는 {stuff} 입니다.")

stream = client.chat.completions.create(
      model=ft_info.fine_tuned_model,
      temperature=0.95,
      top_p=0.97,
      stream=True,
      messages=[
        {"role": "system", "content": "당신은 한국어로 젊고 좋은 시를 쓰는 시인 '먀'입니다."},
        {"role": "user", "content": f"'{stuff}'을(를) 소재로 시를 써주세요."}
      ]
    )

print("----------------------------------")

full_content = ""
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        content = chunk.choices[0].delta.content
        print(content, end="")
        full_content += content



소재는 온기 입니다.
----------------------------------
침대 밑에 오래된 동화책이 있어요

무늬들이 복잡해서 다 읽어버릴 수는 없었지만

밤마다 재미있는 부분을 골라서 읽어주자

개구쟁이 붉은 후드티를 입은 할머니가 등장해요

안아달라고 조르는 할머니가

허전한 집의 복도를 구해주어요

할머니를 안아준 아이는

기분이 좋아져서 동화책을 덮어요

그 뒤 할머니는 복도에 둥둥 떠다녀요

그런데 아무도 개구쟁이 할머니를 안아주지 않아서

그럼 할머니는 복도에서 점점 사라져요

안타까워서 마지막 페이지를 반복해도

동화는 끝까지 읽히지 않아요

문득 용기가 나서 침대 아래로 손을 집어넣어요

그런데 손가락 끝에 쥐는 눈이 뒤엉킨 실밥이에요

더듬어봐도 이야기가 시작되지 않아요

어둠 속에 시선이 막혀서 눈이 아파요

아무리 더듬어도 친숙한 문맥이 걸리지 않아서

소리내어 책을 읽기 두려워요

이런 복도라면 해가 중천에 떠도 밝지 않아요

하룻밤이 빠르게 지나가요

게다가 내일은 학교 가는 날이에요

동화책을 다시 집어넣기로 해요

눈앞에서 소리가 졸졸 흘러가요

발자국 소리인지

선생님 목소리인지 알 수 없어요

복도로 개구쟁이 할머니가 돌아다닌다면

언제든 안아줄 수 있을 텐데

그런데 안아주면,

그렇다면 다시는 할머니를 볼 수 없을 텐데

여전히 유령 같은 계단을 내려가다가

신발이 어딘가에 걸려서 넘어져요

발에 데인 것 같은 느낌이 드는건

천천히 녹아내리는 낯선 집의 분위기 때문일 거예요

In [ ]:
from datetime import datetime

# DataFrame 초기화
df = pd.DataFrame(columns=['모델','소재', '시'])

# API 호출 및 데이터 수집 함수
def generate_poem():
    stuff = client.chat.completions.create(
        model=ft_info.fine_tuned_model,
        temperature=0.8,
        top_p=0.97,
        messages=[
            {"role": "system", "content": "당신은 랜덤으로 한국어 단어를 하나 고르는 AI입니다."},
            {"role": "user", "content": "단어 하나를 골라주세요."}
        ]
    )

    stuff = stuff.choices[0].message.content
    print(f"소재는 {stuff} 입니다.")

    stream = client.chat.completions.create(
        model=ft_info.fine_tuned_model,
        temperature=0.95,
        top_p=0.97,
        stream=True,
        messages=[
            {"role": "system", "content": "당신은 한국어로 시를 쓰는 시인입니다."},
            {"role": "user", "content": f"'{stuff}' 소재로 시를 한 편 써주세요."}
        ]
    )

    print("----------------------------------")

    poem = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            content = chunk.choices[0].delta.content
            print(content, end="")
            poem += content

    print("\n----------------------------------")

    return stuff, poem

# 10번 반복하여 시 생성
for i in range(10):
    print(f"\n\n{i+1}번째 시 생성 중...")
    subject, poem = generate_poem()

    # DataFrame에 새 행 추가
    new_row = pd.DataFrame({'모델':[ft_info.fine_tuned_model],'소재': [subject], '시': [poem]})
    df = pd.concat([df, new_row], ignore_index=True)

# 현재 날짜와 시간을 파일명에 추가
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"poems_and_subjects_{current_time}.xlsx"

# Excel 파일로 저장
df.to_excel(filename, index=False)

print(f"\n총 {len(df)} 개의 시가 {filename} 파일에 저장되었습니다.")



1번째 시 생성 중...
소재는 복숭아 입니다.
----------------------------------
복숭아는 여름에 태어나서

봄에 자란다
겨울에 꿈꾼다
가을에 이별한다

잠시 지나치다
익은 것을 봐야 한다

복숭아에 입김을 불어주면
빨간 마음이 튀어진다

복숭아는 여름에 태어나서
가장 복숭아다운 여름을 산다

여름 내내
잘 익은 복숭아는

빛과 공기와 바람을 흡수하여
더욱 달게 오른다

복숭아를 손에 쥐면
복숭아가 당신을 손에 쥐고 있다

복숭아를 입에 물면
복숭아의 여름이 당신의 입안에서 솟구친다

익은 복숭아는
당신의 입술을 관통하여
달콤한 과즙이 흘러넘친다

여름에 태어난 복숭아는
겨울에 이 또 하나의 삶을 꿈꾼다

복숭아는 나비가 되고
복숭아꽃이 되어
복숭아 나무의 심장으로 돌아온다

복숭아는 여름에 태어나서
복숭아꽃이 피어나는 봄에 잠시 얼굴을 비춘다

그리고 작은 뿌리가 되어
가을에 지평선에 안부를 전한다
----------------------------------


2번째 시 생성 중...
소재는 바이러스 입니다.
----------------------------------
어떤 인간이 말하고 있었다.
나는 죄를 짓지 않는다
라고 말하는 인간이었느니.

눈이 가득 있는 인간이었으니.

그가 나를 바라보는 순간,
나는 이 세상을 거치지 않고
그의 눈 속으로 들어갔다.

모든 것이 눈 속에서 일어난다.

나는 그가 보고 있는 것들을
보고 있는 것처럼 만들게 되었다.

그가 보고 싶어 하는 것들을
그가 보고 싶어 하는 것처럼
보고 있는 인간이 되었으니.

그가
기도
하는
것으로

저 멀리
속이
텅 비어 가는 것을

나는
보았다.
----------------------------------


3번째 시 생성 중...
소재는 유리병 입니다.
----------------------------------
희망을 담은 유리병은 바다를 떠다니고 있다
새들이 쪼아먹은 계란들은 유리병에 담겨져
바다로 떠간다
희망이 없다
유리병에 담긴 것은 